# Lab3 - Second part: Machine learning in Spark
## Task 2 - SVM with L-BFGS

Repeat task 1 running SVM with the L-BFGS optimization algorithm. Setting the maximum number of iterations to 30 is fine for the purpose of this assignment.

In [ ]:
TODO

## Does L-BFGS perform better than SGD for this training set?

TODO